# 📘 10_load_videos_static_to_bigquery

### 🎯 Objetivo

Este notebook no transforma datos.  
Su única responsabilidad es: Cargar df_videos_static (generado en el notebook 04) hacia BigQuery como tabla histórica.  
Destino: youtube-datasets-360.angelgarciadatablog.videos_static

In [ ]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

In [ ]:
PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "videos_static"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)

## 🧱 Cargar snapshot desde Parquet (temporal)

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

file_path = PROCESSED_PATH / "videos_static.parquet"

print("Ruta:", file_path)
print("Existe:", file_path.exists())

df_videos_static = pd.read_parquet(file_path)

df_videos_static.head()


In [ ]:
df_videos_static.dtypes

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 




In [ ]:
from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("video_id", "STRING"),
    SchemaField("channel_id", "STRING"),
    SchemaField("title", "STRING"),
    SchemaField("description", "STRING"),
    SchemaField("category_id", "STRING"),
    SchemaField("published_at", "TIMESTAMP"),
    SchemaField("duration_seconds", "INT64"),
    SchemaField("thumbnail_url", "STRING"),
    SchemaField("video_url", "STRING"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("Tabla ya existe.")
    
except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)
    client.create_table(table)
    print("Tabla creada.")


## 📌 Cargar datos del parquet a big query

In [ ]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE TRUNCATE = sobreescribe los datos
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df_videos_static,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Tabla videos_static reemplazada correctamente.")
